# Lecture 2
こちらは __宇宙ビッグデータの画像__ を7つのクラス「Bare (裸地), Built-up (建物), Cropland (農地), Forest (森), Grassland (草原), Paddy_field (水田), Water_bodies (水域)」に分類するプログラムのサンプルです．
内容については，Lecture1とほとんど同じですので，こちらも __講義で説明した改良部分 (ハイパーパラメータ，モデル構造)__を変更し，コンペティションで上位を目指しましょう！

In [ ]:
# プログラムで利用する各種パッケージの定義です
import os
import re
import pickle
from PIL import Image
from datetime import datetime
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, ReLU, Softmax
from keras.saving import save_model
from keras.optimizers import Adam
from keras.metrics import F1Score
from keras.utils import to_categorical, plot_model

In [ ]:
# # サーバから宇宙ビッグデータをダウンロードします
!wget -N --http-user=aistepkanazawa --http-password=spaceai "https://ai-step.ec.t.kanazawa-u.ac.jp/competition_02/data/competition02_32_60_7_dataset.zip"
!unzip -o -q "competition02_32_60_7_dataset.zip"

In [ ]:
# 訓練データ，テストデータのファイル名を設定
base_dir = ""
train_pkl = os.path.join(base_dir, "fukui_train_32_60_ver2.pkl")
test_pkl = os.path.join(base_dir, "kanazawa_test_32_60_ver2.pkl")

# 出力ファイル名
dt_str = datetime.now().strftime("%y%m%d_%H%M%S")
model_name = "model_{}.keras".format(dt_str)
result_csv_name = "result_csv_{}.csv".format(dt_str)
result_img_name = "result_img_{}.png".format(dt_str)

In [ ]:
# 画像のサイズ (横幅) 単位：ピクセル
IMG_WIDTH = 32
# 画像のサイズ (縦幅) 単位：ピクセル
IMG_HEIGHT = 32

# 今回は7種類に分類を行います (classesには「7」が入る)
#   Bare (裸地), Built-up (建物), Cropland (農地), Forest (森), Grassland (草原), Paddy_field (水田), Water_bodies (水域)
classes = 7

# モデルが出力する分類結果 (数値)と，名前を紐づける
#   Bare = 0, Built-up = 1, Cropland = 2, Forest = 3, Grassland = 4, Paddy_field = 5, Water_bodies = 6
class_names = ["Bare", "Built-up", "Cropland", "Forest", "Grassland", "Paddy_field", "Water_bodies"]

## ハイパーパラメータの設定
こちらは，学習に用いる各種パラメータ (ハイパーパラメータと呼ばれる)を設定しています．

- **バッチサイズ**
    - 主に**2のn乗**の値を設定します
    - こちらの値が大きい方が並列で学習しますので，速度が向上します
        - 大きくしすぎると，メモリが足りなくなってエラー終了しますので注意してください
    - 例：2, 4, 8, 16, 32, ...
- **エポック数**
    - 繰り返し学習をする回数になります．精度が落ち着くまでの回数を指定しましょう．
    - 1エポックは，すべての学習データを使って1回学習しています．
- **学習率**
    - 重みをどの程度の大きさで変更するか？を指定する値になります．
    - 主に0.1以下で，0.01，0.001など，非常に小さな値を指定します
    - 学習を進めても精度が変わらない場合は，こちらの値を増減させてみてください．

In [ ]:
###################################################################################
# こちらの項目は「ハイパーパラメータ」と呼ばれる項目になります
# 値を変更してモデルの精度を向上させてみましょう！
###################################################################################
# バッチサイズ (並列して学習を実施する数)
batch_size = 256

# エポック数 (学習を何回実施するか？という変数)
epochs = 20

# 学習率 (重みをどの程度変更するか？)
learning_rate = 0.0001

In [ ]:
# 訓練データと検証データに分割を行う関数
def train_val_split(x, y, ratio=0.7, shuffle=False):
    train_idx_list = []
    val_idx_list = []

    for i in range(classes):
        all_data = np.where(y == i)[0]
        train_idx, val_idx = np.split(all_data, [int(all_data.size * ratio)])
        train_idx_list.extend(train_idx)
        val_idx_list.extend(val_idx)

    # シャッフルの指定があれば，データをシャッフルする
    if shuffle:
        np.random.shuffle(train_idx_list)
        np.random.shuffle(val_idx_list)

    return (np.array(x[train_idx_list], dtype=np.float32),
            np.array(y[train_idx_list], dtype=np.uint8),
            np.array(x[val_idx_list], dtype=np.float32),
            np.array(y[val_idx_list], dtype=np.uint8))

In [ ]:
# 訓練データを読み込みます
with open(train_pkl, 'rb') as f:
    (train_img, train_label) = pickle.load(f)

# 訓練データと検証データに分割を行います
#  ratio=0.7で，訓練データの割合は70%としています
#  shuffle=Falseで，データのシャッフルは行いません (毎回同じデータが訓練・検証データに分割されます)
train_x, train_y, val_x, val_y = train_val_split(train_img, train_label, ratio=0.7, shuffle=False)

# データの前処理
train_x = train_x / 255.0
train_y = to_categorical(train_y, classes)

# データの前処理
val_x = val_x / 255.0
val_y = to_categorical(val_y, classes)

## モデル構造 (特徴抽出器)の定義
モデルの構造について，こちらで定義しています．
基本的には，以下の順序で層を重ねることでモデルを構築していきます．

### 各層について
- __畳み込み層 (Conv2D)__
    - filters  ：出力するフィルタ (チャネル)の枚数を指定します
    - kernel_size  ：畳み込みのカーネルサイズを指定します (奇数にすることがほとんどです 3, 5, 7, ...)
- __活性化関数 (ReLU)__
    - 活性化関数はいくつかありますが，畳み込みのあとは「ReLU」関数で問題ないでしょう
- __プーリング層 (MaxPool2D)__
    -  __2__ を指定すると，画像サイズが1/2になり， __4__ を指定すると，画像サイズが1/4になります
    - 基本的には __2__ のみを指定することが多いです

### 層の組み合わせ
以下のように組み合わせて構築することが多いです．
**畳み込みと活性化関数は，必ずセットで**利用するようにしましょう
- 畳み込み - 活性化関数 - プーリング層
- 畳み込み - 活性化関数 - 畳み込み - 活性化関数 - プーリング層
- 畳み込み - 活性化関数 - ... - プーリング層

### 畳み込みのフィルタ (チャネル)とプーリングの関係
また，プーリング層を挟んで画像のサイズを半分にすると，その次から畳み込みのフィルタ数を2倍にするのが基本的な設定です．

    Conv2D(filters=32)
    ReLU()
    MaxPool2D(2)

    Conv2D(filters=64)
    ReLU()
    MaxPool2D(2)

    Conv2D(filters=128)
    ...

In [ ]:
# モデルの構築
#  この「model」という変数に，構築するモデルのすべての情報が入ります
model = tf.keras.Sequential()

# モデルの編集 (特徴抽出器)
# 編集場所はここから！
###################################################################################
# 畳み込み - 活性化関数 (ReLU) - プーリング
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='same'))
model.add(ReLU())
model.add(MaxPool2D(2))

# 畳み込み - 活性化関数 (ReLU) - プーリング
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='same'))
model.add(ReLU())
model.add(MaxPool2D(2))
###################################################################################
# ここまで！

# 分類器 (こちらは編集しない！)
model.add(Flatten())
model.add(Dense(classes))
model.add(Softmax())

In [ ]:
# 損失関数 (1 - macro-F1を用いて損失を計算する)
@tf.function
def macro_soft_f1(y, y_pred):
    y = tf.cast(y, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    tp = tf.reduce_sum(y_pred * y, axis=0)
    fp = tf.reduce_sum(y_pred * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_pred) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1
    loss = tf.reduce_mean(cost)
    return loss

# 最適化関数の設定
#  "パラメータをどのように更新していくか？"という設定項目になります (学習率をこちらで使っています)
optimizer = Adam(learning_rate=learning_rate)

# 学習が実施できるように，モデルの設定を完了します
#  loss=macro_soft_f1 で，損失関数として 1 - macro-F1を設定しています
#  metricsに「F1Score」を設定して，macro-F1で評価した結果を出力します
model.compile(optimizer=optimizer,
              loss=macro_soft_f1,
              metrics=["accuracy", F1Score(average="macro", threshold=0.5)])

model.build(input_shape=(batch_size, IMG_HEIGHT, IMG_WIDTH, 3))

# 確認のため，モデルの構造を表示してみます
plot_model(model, show_shapes=True, expand_nested=True)

In [ ]:
# 学習を実施します
#  訓練データと検証データをそれぞれ設定しています
history = model.fit(x=train_x, y=train_y, validation_data=(val_x, val_y),
                    epochs=epochs, batch_size=batch_size)

# モデルの保存を行います
#   ファイル名は「model_(日時).keras」として，保存されます
save_model(model, model_name, overwrite=True, save_format="keras")

以下は，コンペティションで提出するデータを作成するプログラムです．
また，テストデータに対して，土地被覆図の画像についても作成します．

In [ ]:
# 混同行列と，macro-F1を出力する関数です
def create_cm_f1(x, y_true):
    y_pred = []
    for i in range(0, len(x), batch_size):
        # データをバッチ化して保存
        batch_img = x[i:i+batch_size]

        # 予測の実施
        predictions = model.predict(batch_img, verbose=0)
        predicted_classes = np.argmax(predictions, axis=1)

        # 結果をリストに保存
        y_pred.extend(predicted_classes)

    # データの整形
    y_pred = np.asarray(y_pred)
    if y_true.ndim != 1:
        y_true = np.asarray(y_true).argmax(axis=1)

    # 混同行列を表示
    cm = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=list(range(0, classes)))

    cm_df = pd.DataFrame(data=cm, index=class_names, columns=class_names)
    sns.heatmap(cm_df, annot=True, fmt='d', cmap="Blues")

    macro_f1 = precision_score(y_true=y_true, y_pred=y_pred, average='macro', zero_division=0.0)
    print("F1-Score (macro-F1) = {}".format(macro_f1))

In [ ]:
# 検証データに対する混同行列を作成して，macro-F1を確認する
create_cm_f1(train_x, train_y)

In [ ]:
# 検証データに対する混同行列を作成して，macro-F1を確認する
create_cm_f1(val_x, val_y)

In [ ]:
# テストデータを読み込んで実施してみる
with open(test_pkl, 'rb') as f:
    (test_img, file_name_list) = pickle.load(f)
# テストデータの前処理
test_img = test_img / 255.0

data_list = []
# バッチごとに処理して，CSVを作成する
for i in range(0, len(test_img), batch_size):
    # データをバッチ化して保存
    batch_img = test_img[i:i+batch_size]
    batch_file_names = file_name_list[i:i+batch_size]

    # 予測の実施
    predictions = model.predict(batch_img, verbose=0)
    predicted_classes = np.argmax(predictions, axis=1)

    # 結果をリストに保存
    batch_result = list(zip(batch_file_names, predicted_classes))
    data_list.extend(batch_result)
df = pd.DataFrame(data_list)

#################################################################################
# テストデータに対する分類結果をCSVファイルに保存します
#  ファイル名は，「result_csv_(日時).csv」となります
#  このファイルを提出してください！！！
#################################################################################
df.to_csv(result_csv_name, index=False, header=False)

テストデータに対する分類結果を利用して，土地被覆図を作成して表示してみます

In [ ]:
# 土地被覆図の画素について辞書作成 (BGR)
dic = {
    0: (0, 100, 128),
    1: (0, 0, 255),
    2: (191, 193, 255),
    3: (0, 128, 0),
    4: (0, 255, 255),
    5: (255, 128, 0),
    6: (100, 0, 0),
}

cell_size = 32

# CSVファイルを読み込む
df = pd.read_csv(result_csv_name, header=None)

ref_file = df[0][0]
# グリッドのサイズを取得
cols = int(re.findall(r"\d+", str(ref_file))[-3])
rows = int(re.findall(r"\d+", str(ref_file))[-4])

# グリッド画像のサイズを計算
width = cell_size * cols
height = cell_size * rows

# 空の画像を作成（白色の背景）
inference_img = np.ones((height, width, 3), dtype=np.uint8) * 255

# 各セルにデフォルトの値を設定
default_img = np.ones((cell_size, cell_size, 3), dtype=np.uint8) * 255

for i in range(len(df)):
    filename = df.iloc[i, 0]
    cell_value = int(df.iloc[i, 1])

    match = re.search(r'_(\d+)_(\d+).png$', filename)
    if match:
        row = int(match.group(1))
        col = int(match.group(2))

        if cell_value in dic:
          img = np.ones((cell_size, cell_size, 3), dtype=np.uint8) * 255
          img[:,:] = dic[cell_value]

        else:
          img = default_img

        inference_img[row * cell_size:(row + 1) * cell_size, col * cell_size:(col + 1) * cell_size] = img

img_rgb = inference_img[:, :, [2, 1, 0]]

#################################################################################
# 画像を保存します
#  ファイル名は，「result_img_(日時).png」となります
#################################################################################
pil_image = Image.fromarray(img_rgb)
pil_image.save(result_img_name)

# 結果の表示
ax = plt.gca()
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
plt.imshow(pil_image)